# 🎯 ES診断ツール - Google Colab起動用

このノートブックを使用して、ES診断ツールをGoogle Colab上で起動します。

## 📋 実行手順

1. **セル1**: パッケージのインストール
2. **セル2**: GitHubリポジトリをクローン
3. **セル3**: CSVデータの準備（Google Drive / アップロード）
4. **セル4**: アプリケーションの起動

---

## セル1: 必要なパッケージをインストール

In [ ]:
# 必要なパッケージをインストール
!pip install flask flask-cors pandas numpy scikit-learn -q

# sentence-transformers をインストール（セマンティック検索用）
print("🔧 sentence-transformersをインストール中...")
!pip install sentence-transformers -q
print("✅ sentence-transformersのインストールが完了しました")

# pyngrokを個別にインストール（タイムアウト対策）
!pip install pyngrok -q --default-timeout=100

print("✅ 全パッケージのインストールが完了しました")
print("\n📦 インストールされたパッケージ:")
print("  - Flask (Webアプリケーション)")
print("  - pandas (データ処理)")
print("  - numpy (数値計算)")
print("  - scikit-learn (TF-IDF、機械学習)")
print("  - sentence-transformers (セマンティック検索用BERT)")
print("  - pyngrok (公開URL作成)")

## セル2: GitHubリポジトリをクローン

In [2]:
# リポジトリをクローン
!git clone https://github.com/ryosuke-fujii/es.git

# カレントディレクトリを変更
%cd es

print("✅ リポジトリのクローンが完了しました")
print("\n📁 ディレクトリ構造:")
!ls -la

Cloning into 'es'...
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 27 (delta 6), reused 26 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (27/27), 21.09 KiB | 2.64 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/es
✅ リポジトリのクローンが完了しました

📁 ディレクトリ構造:
total 56
drwxr-xr-x 7 root root  4096 Nov  4 15:37 .
drwxr-xr-x 1 root root  4096 Nov  4 15:37 ..
-rw-r--r-- 1 root root 10402 Nov  4 15:37 COLAB_GUIDE.md
drwxr-xr-x 2 root root  4096 Nov  4 15:37 data
drwxr-xr-x 8 root root  4096 Nov  4 15:37 .git
-rw-r--r-- 1 root root   486 Nov  4 15:37 .gitignore
drwxr-xr-x 2 root root  4096 Nov  4 15:37 notebooks
-rw-r--r-- 1 root root  6790 Nov  4 15:37 README.md
-rw-r--r-- 1 root root    94 Nov  4 15:37 requirements.txt
drwxr-xr-x 2 root root  4096 Nov  4 15:37 src
drwxr-xr-x 2 root root  4096 Nov  4 15:37 templates


## セル3: CSVデータの準備

以下の2つの方法から選択してください：
- **方法A**: Google Driveから読み込む（推奨）
- **方法B**: 直接アップロード

In [3]:
# ============================================
# 方法A: Google Driveから読み込む（推奨）
# ============================================

from google.colab import drive
import os

# Google Driveをマウント
drive.mount('/content/drive')

# CSVファイルのパスを指定（あなたのパスに変更してください）
csv_path = "/content/drive/MyDrive/企画・マーケチーム/gaxi_自動化/就活会議/詳細スクレイピング/インターン__ES_syukatsu-kaigi.jp-から詳細をスクレイプします--3--2025-10-31 (1).csv"

# ファイルが存在するか確認
if os.path.exists(csv_path):
    print(f"✅ ファイルを発見: {csv_path}")
else:
    print(f"❌ ファイルが見つかりません: {csv_path}")
    print("\n💡 ヒント: ファイルブラウザで目的のファイルを右クリック → 'パスをコピー'")

Mounted at /content/drive
✅ ファイルを発見: /content/drive/MyDrive/企画・マーケチーム/gaxi_自動化/就活会議/詳細スクレイピング/インターン__ES_syukatsu-kaigi.jp-から詳細をスクレイプします--3--2025-10-31 (1).csv


In [ ]:
# ============================================
# 方法B: 直接アップロード
# ============================================

from google.colab import files

# ファイルをアップロード
uploaded = files.upload()

# アップロードされたファイル名を取得
csv_path = list(uploaded.keys())[0]
print(f"✅ ファイルをアップロードしました: {csv_path}")

## セル4: アプリケーションを起動

In [ ]:
import sys
import os
import threading
import time

# srcディレクトリをPythonパスに追加
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

# アプリケーションをインポート
from app import app, load_csv_data

# ============================================
# ngrokの認証トークンを設定
# ============================================
# https://dashboard.ngrok.com/ でトークンを取得してください

NGROK_TOKEN = "YOUR_NGROK_TOKEN"  # ここにあなたのngrokトークンを入力

# ngrokインポートとセットアップ（リトライ付き）
print("🔧 ngrokをセットアップ中...")
for attempt in range(3):
    try:
        from pyngrok import ngrok
        ngrok.set_auth_token(NGROK_TOKEN)
        print("✅ ngrok認証トークンを設定しました")
        break
    except Exception as e:
        if attempt < 2:
            print(f"⚠️ リトライ {attempt + 1}/3...")
            time.sleep(5)
        else:
            print(f"❌ ngrokのセットアップに失敗しました: {e}")
            print("\n💡 解決方法:")
            print("1. ランタイムを再起動してもう一度試す")
            print("2. セル1を再実行してパッケージを再インストール")
            raise

# ============================================
# CSVデータを読み込み
# ============================================
print("\n📂 CSVデータを読み込み中...")
print("⏳ 初回実行時はSentence-BERTモデルのダウンロードに数分かかります...")
load_csv_data(csv_path)
print("✅ データの読み込みが完了しました")

# ============================================
# Flaskアプリをバックグラウンドで起動
# ============================================
def run_flask():
    app.run(port=5000)

flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

print("\n🚀 Flaskアプリを起動しました (ポート5000)")

# ============================================
# ngrokトンネルを作成
# ============================================
time.sleep(2)  # Flaskの起動を待つ

print("\n🌐 ngrokトンネルを作成中...")
try:
    public_url = ngrok.connect(5000)
    
    print("\n" + "="*60)
    print("🎉 アプリケーションが起動しました！")
    print("="*60)
    print(f"\n🌐 公開URL: {public_url}")
    print("\n💡 上記のURLをクリックしてアプリにアクセスしてください")
    print("\n⚠️ このノートブックを閉じるとアプリも停止します")
    print("\n🔍 搭載機能:")
    print("  - TF-IDFキーワードマッチング")
    print("  - セマンティック検索（BERT）で意味的類似性を理解")
    print("  - STAR構造分析でES品質を評価")
    print("="*60)
except Exception as e:
    print(f"\n❌ ngrokトンネルの作成に失敗しました: {e}")
    print("\n💡 トラブルシューティング:")
    print("1. NGROK_TOKENが正しく設定されているか確認")
    print("2. https://dashboard.ngrok.com/ でトークンを再確認")
    print("3. ランタイムを再起動して最初からやり直す")
    raise


## 🔧 トラブルシューティング

### sentence-transformersのインストールが遅い
```
初回インストール時に時間がかかる場合があります
```

**対処法：**
- sentence-transformersは大きなライブラリ（PyTorchを含む）のため、インストールに5-10分かかることがあります
- 進行中のメッセージが表示されていれば正常です
- 完了を待ってください

### Sentence-BERTモデルのダウンロードが遅い
```
初回実行時にモデルのダウンロードに時間がかかる
```

**対処法：**
- 初回実行時のみ、モデル（約400MB）がダウンロードされます
- 2回目以降はキャッシュされたモデルを使用するため高速です
- セル4で「⏳ 初回実行時はSentence-BERTモデルのダウンロードに数分かかります...」と表示されていれば正常です

### ngrokダウンロードのタイムアウトエラー
```
TimeoutError: The read operation timed out
```

**対処法：**
1. ランタイムを再起動して最初から実行
2. 別の時間帯に試す（ネットワーク混雑時を避ける）
3. セル1で個別にインストール：
   ```python
   !pip install pyngrok --default-timeout=100 -q
   ```

### CSVファイルが見つからない
- セル3でファイルパスを確認してください
- Google Driveの場合、マウントが正しく行われているか確認してください
- ファイルブラウザで実際のパスをコピーして使用

### ngrok認証エラー
```
Invalid credentials
```

**対処法：**
- セル4の `NGROK_TOKEN` を正しく設定してください
- https://dashboard.ngrok.com/ でトークンを再確認
- トークンの前後にスペースや改行がないか確認

### メモリエラー
```
MemoryError
```

**対処法：**
- CSVファイルが大きすぎる可能性があります
- sentence-transformersも大量のメモリを使用します
- Google Colab Proの使用を検討
- データをフィルタリング：
   ```python
   import pandas as pd
   df = pd.read_csv(csv_path)
   df = df.tail(10000)  # 最新1万件のみ
   df.to_csv('temp.csv', index=False)
   csv_path = 'temp.csv'
   ```

### インポートエラー
```
ModuleNotFoundError: No module named 'app'
```

**対処法：**
- セル2でリポジトリのクローンが成功しているか確認
- `%cd es` が実行されているか確認
- セル4を再実行

### セマンティック検索が使えない場合
```
⚠️ sentence-transformersがインストールされていません。TF-IDFのみを使用します。
```

**対処法：**
- アプリは動作しますが、意味的類似性は使われません
- セル1を再実行してsentence-transformersをインストール
- ランタイムを再起動してすべてのセルを順番に実行